In [1]:
import networkx as nx
import d3networkx as d3nx
from d3graph import D3Graph, D3DiGraph
import random
import math

This next line starts up the visualizer. It will start some background code that sends data to the visualizer and then it will open a new browser window where the visualizer will live. Once you have the visualizer running, you can leave it running for the entire session, so don't re-run this block. If you close the `visualizer.html` (or hit refresh), you will need to reestablish this connection. In this case, you should click the refresh button in the Jupyter notebook (not for the webpage) to restart the kernel (which will clear your variables and Python environment).

In [2]:
d3 = await d3nx.create_d3nx_visualizer(event_delay=0.03, interactive=False, canvas_size=(1200,800),
                        node_dstyle=d3nx.node_style(size=4,fill='#111111'), 
                        node_hstyle=d3nx.node_style(fill='#EB4343'),
                        edge_hstyle=d3nx.edge_style(stroke='#EB4343',stroke_width=5))

websocket server started...networkx connected...visualizer connected...

We will learn about this kind of graph, but it is a way of generating a type of graph, in this case with `N` nodes. Notice that the graph that we use needs to be a `D3Graph`, which we can make by simply creating it with the `nx.Graph` object.

In [3]:
N = 100
G = nx.erdos_renyi_graph(N,0.03)
G = D3Graph(G)
print(G.number_of_edges())
d3.set_graph(G)

122


Since interactive was set to `False`, we have yet to see the graph appear, so we call update to make it happen.

In [4]:
d3.update()

Now we change the visualizer to be interactive.

In [5]:
d3.set_interactive(True)

Here, I'm showing how you can explicitly position the location of nodes in the network, rather than relying on the *force layout* that is typically used. This is fun to watch because we've made the visualizer interactive now!

In [6]:
d3.set_title('Positioning Nodes...')
for nidx, n in enumerate(G.nodes()):
    x = 250 + 230 * math.cos(math.radians(360.0*nidx/G.number_of_nodes()))
    y = 300 + 150 * math.sin(math.radians(360.0*nidx/G.number_of_nodes()))
    d3.position_node(n,x,y)

Now I'm going to randomly highlight about 10% of the edges (`p=0.1`). Notice that `highlight_edges` and `highlight_nodes` are both plural, so if you are highlighting a single edge, you need to put it in a list/set. 

In [7]:
d3.set_title('Highlighting Edges...')
p = 0.1
active_nodes = set()
for e in G.edges():
    if random.random() <= p:
        u,v = e
        active_nodes.add(u)
        active_nodes.add(v)
        d3.highlight_edges([e])

Now we go back and highlight all the nodes.

In [8]:
d3.highlight_nodes(active_nodes)

Here we're adding a new node and giving it a unique style using the `stylize_node` method. You can do the same thing with edges.

In [9]:
d3.set_title('Adding and styling a single node...')
G.add_node('new node')
G.add_edge(0,'new node')
d3.stylize_node('new node', d3nx.node_style(fill='#FFA929',size=12))

Here we show how to transition from one network to another using the `clear` command, then setting the graph with `set_graph`. Since the new network is large enough, we set interactive to be `False`. Notice that we still need this to be a `D3DiGraph` not just a `nx.DiGraph` - this time we can do it all in one line, which is convenient.

In [10]:
d3.set_title('Switching to a directed network...')
d3.clear()
G = D3DiGraph(nx.scale_free_graph(100))
d3.set_interactive(False)
d3.set_graph(G)
d3.update()